In [ ]:
import requests
from getpass import getpass

In [ ]:
data = {
    "email":input('email:'),
    "password":getpass('password:')
}

In [ ]:
response = requests.post('https://bootcampspot.com/api/instructor/v1/login', json=data)

In [ ]:
params = {
    "Content-Type":"application/json",
    "authToken":response.json()['authenticationInfo']['authToken']
}

In [ ]:
me = requests.get('https://bootcampspot.com/api/instructor/v1/me', params=params)

In [ ]:
courseId = me.json()['enrollments'][0]['courseId']

In [ ]:
json = {
    'courseId':courseId
}

In [ ]:
attendance = requests.post('https://bootcampspot.com/api/instructor/v1/attendance', params=params, json=json)

In [ ]:
grades = requests.post('https://bootcampspot.com/api/instructor/v1/grades', params=params, json=json)

In [ ]:
json = {
    'enrollmentId':me.json()['enrollments'][0]['id']
}
assignments = requests.post('https://bootcampspot.com/api/instructor/v1/assignments', params=params, json=json)

In [ ]:
meh = []
for d in response.json()['enrollments']:
    for k in ['courseId', 'active']:
        print(d.get(k))

In [ ]:
[d.get(k) for d in response.json()['enrollments'] for k in ['courseId', 'course']]

In [ ]:
sg = {}
for s in grades.json():
    a = {
        'title':s['assignmentTitle'],
        'submitted':s['submitted'],
        'grade':s['grade']
    }
    if s['studentName'] not in sg:
        sg[s['studentName']] = [a]
    else:
        sg[s['studentName']].append(a)

In [ ]:
grades.json()

In [ ]:
for x in me.json()['enrollments']:
    for y in x:
        if y in ['id','courseId']:
            print(x[y])

In [ ]:
me.json()['enrollments'][0]['id']

In [ ]:
assignments